In [1]:
import pandas as pd
from sqlalchemy import create_engine
from clickhouse_driver import Client

In [2]:
# MySQL connection settings
DB_USER = 'root'
DB_PASS = 'root'
DB_HOST = 'localhost'
DB_PORT = '3306'
DB_NAME = 'ecommerce'

# Create MySQL connection string
connection_str = f"mysql+pymysql://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_NAME}"

# Create SQLAlchemy engine
engine = create_engine(connection_str)

In [3]:
# Query orders table
orders_query = "SELECT * FROM orders"
orders_df = pd.read_sql(orders_query, engine)

# Query products table
products_query = "SELECT * FROM products"
products_df = pd.read_sql(products_query, engine)

# Query order_items table
order_items_query = "SELECT * FROM order_items"
order_items_df = pd.read_sql(order_items_query, engine)

In [4]:
# Merge order_items with products to get the price
merged_df = pd.merge(order_items_df, products_df, left_on='product_id', right_on='id', how='left')

# Merge with orders table
final_merged_df = pd.merge(orders_df, merged_df, left_on='id', right_on='order_id', how='left')

# Convert order_date to datetime
final_merged_df['order_date'] = pd.to_datetime(final_merged_df['order_date'])

# Aggregate data
daily_sales_df = final_merged_df.groupby('order_date').agg({
    'quantity': 'sum',
    'price': 'sum'
}).reset_index()

daily_sales_df.head()

,order_date,quantity,price
0,2022-01-01,63,30
1,2022-01-02,66,60
2,2022-01-03,69,90
3,2022-01-04,72,120
4,2022-01-05,75,150


In [10]:
from clickhouse_driver import Client

# Connect to ClickHouse
ch_client = Client(host='localhost', port=9000, user='default', password='')

# Convert DataFrame to list of tuples
data = [tuple(row) for row in daily_sales_df.values]

# Define ClickHouse query to insert data
query = "INSERT INTO daily_sales (order_date, quantity, total_sales) VALUES"

# Execute the query to insert data
ch_client.execute(query, data)

20